In [1]:
from chronos import BaseChronosPipeline, Chronos2Pipeline
import pandas as pd
import os
from dotenv import load_dotenv
import requests
from utils import upload
import dropbox
from pandas.tseries.frequencies import to_offset
import holidays

load_dotenv()

/home/dhopkins/apps/ed-flow-2023/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
import torch
torch.cuda.set_device(6)

# Load the Chronos-2 pipeline
# GPU recommended for faster inference, but CPU is also supported
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="cuda"
    # device_map="cpu"
)

In [3]:
def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

def add_holiday_flags(
    df: pd.DataFrame,
    ts_col: str = "ds",
    local_tz: str = "America/Montreal",
    observed: bool = True,
    include_names: bool = False,
) -> pd.DataFrame:
    """
    Adds boolean columns:
      • is_qc_holiday       — Québec public holiday (CA-QC)
      • is_jewish_holiday   — Israeli public/Jewish holiday (IL)
    Optionally adds:
      • qc_holiday_name
      • jewish_holiday_name

    Notes:
      • Holiday checks are date-based (00:00–24:00 local calendar date),
        not sundown-to-sundown observance.
      • NaT timestamps are ignored gracefully.
    """
    out = df.copy()

    # 1) Parse to datetime
    out[ts_col] = pd.to_datetime(out[ts_col], errors="coerce")

    # 2) Get the calendar DATE to use for holiday lookup
    #    - If tz-aware: convert to Montreal then take .date
    #    - If naive: assume values already represent local Montreal wall-clock; just take .date
    if getattr(out[ts_col].dt, "tz", None) is not None:
        dates_for_calendar = out[ts_col].dt.tz_convert(local_tz).dt.date
    else:
        dates_for_calendar = out[ts_col].dt.date

    # 3) Build a SAFE integer year range for the holiday objects
    years_series = pd.Series(dates_for_calendar)
    years_series = years_series.dropna().map(lambda d: int(pd.Timestamp(d).year))
    if years_series.empty:
        raise ValueError("No valid datetimes found to extract holiday years.")
    years = list(range(int(years_series.min()), int(years_series.max()) + 1))

    # 4) Construct holiday calendars
    qc_holidays = holidays.Canada(subdiv="QC", years=years, observed=observed)
    il_holidays = holidays.Israel(years=years, observed=observed)

   # 5) Flag membership
    out["is_qc_holiday"] = [ ("yes" if d in qc_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                             for d in dates_for_calendar ]
    out["is_jewish_holiday"] = [ ("yes" if d in il_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                                 for d in dates_for_calendar ]

    if include_names:
        out["qc_holiday_name"] = [ qc_holidays.get(d, "no") if pd.notna(pd.Timestamp(d)) else "no"
                                   for d in dates_for_calendar ]
        out["jewish_holiday_name"] = [ il_holidays.get(d, "no") if pd.notna(pd.Timestamp(d)) else "no"
                                       for d in dates_for_calendar ]

    return out

shift_types_dict = {'W1':'flow',
 'X1':'pod',
 'X3':'pod',
 'X4':'vertical',
 'X2':'vertical',
 'WOC1':'oncall',
 'WOC2':'oncall',
 'WOC3':'oncall',
 'X5':'pod',
 'W3':'overlap',
 'Y1':'pod',
 'Y3':'pod',
 'Y4':'vertical',
 'Y2':'vertical',
 'Y5':'pod',
 'Z1':'night',
 'Z2':'night',
 'D1':'pod',
 'R1':'pod',
 'P1':'vertical',
 'D2':'vertical',
 'OC1':'oncall',
 'OC2':'oncall',
 'V1':'flow',
 'A1':'pod',
 'G1':'vertical',
 'E1':'pod',
 'R2':'pod',
 'A2':'pod',
 'P2':'vertical',
 'E2':'vertical',
 'N1':'night',
 'N2':'night',
 'L2':'overlap',
 'L4':'overlap',
 'H1':'teaching',
 'B1':'vertical',
 'L1':'overlap',
 'W5':'overlap',
 'L6':'overlap',
 'B2':'vertical'}

In [21]:
# Load hourly data
df = pd.read_csv(
    'https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id'] = 'jgh'
df.tail()
hourly_report_df = df.copy()

In [5]:
# Load shift data
all_shifts_df = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_df['shift_start'] = pd.to_datetime(all_shifts_df['shift_start']).dt.round('h')
all_shifts_df['shift_end'] = pd.to_datetime(all_shifts_df['shift_end']).dt.round('h')
all_shifts_df['shift_type'] = all_shifts_df['shift_short_name'].map(shift_types_dict)
all_shifts_df.tail()

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend,shift_type
29962,88860,1,66,NaN,NaN,Wayne,Choi,1,NaN,Jewish General Hospital,...,2026-02-24 16:00:00,2026-02-25 00:00:00,8.00,2/24/2026 16:00,2/25/2026 00:00,8.00,1,0,0,vertical
29963,88858,1,18,NaN,NaN,Michael,Engo,1,NaN,Jewish General Hospital,...,2026-02-24 16:00:00,2026-02-25 00:00:00,8.00,2/24/2026 16:00,2/25/2026 00:00,8.00,1,0,0,pod
29964,88861,1,80,NaN,NaN,Natasha,Poirier,1,NaN,Jewish General Hospital,...,2026-02-24 18:00:00,2026-02-25 02:00:00,8.00,2/24/2026 18:00,2/25/2026 02:00,8.00,1,0,0,vertical
29965,88211,1,25,NaN,NaN,Devin,Hopkins,1,NaN,Jewish General Hospital,...,2026-02-25 00:00:00,2026-02-25 08:00:00,8.08,2/24/2026 23:45,2/25/2026 07:50,8.08,1,1,0,night
29966,88862,1,61,NaN,NaN,Alexander,Hart,1,NaN,Jewish General Hospital,...,2026-02-25 00:00:00,2026-02-25 08:00:00,8.08,2/24/2026 23:45,2/25/2026 07:50,8.08,1,1,0,night


In [6]:
# Create hourly rows
# We'll use a list comprehension to generate the range for each row
expanded_rows = []
for _, row in all_shifts_df.iterrows():
    # Create range. inclusive='left' means [start, end)
    # If start == end (e.g. 0 length shift after rounding), it will be empty, which is correct
    hours = pd.date_range(row['shift_start'], row['shift_end'], freq='h', inclusive='left')
    for h in hours:
        expanded_rows.append({
            'ds': h,
            'user': row['first_name']+row['last_name'],
            'shift_type': row['shift_type'],
            'shift_short_name': row['shift_short_name']
        })

expanded_df = pd.DataFrame(expanded_rows)

expanded_df.tail()

,ds,user,shift_type,shift_short_name
272944,2026-02-25 03:00:00,AlexanderHart,night,N2
272945,2026-02-25 04:00:00,AlexanderHart,night,N2
272946,2026-02-25 05:00:00,AlexanderHart,night,N2
272947,2026-02-25 06:00:00,AlexanderHart,night,N2
272948,2026-02-25 07:00:00,AlexanderHart,night,N2


In [7]:
# Pivot
# index=timestamp, columns=user_id, values=shift_type
hourly_shifts_by_user_df = expanded_df.pivot_table(
    index='ds', 
    columns='user', 
    values='shift_type', 
    aggfunc='first' # In case of duplicates, take the first
)

# Fill NaNs
hourly_shifts_by_user_df = hourly_shifts_by_user_df.fillna('NotWorking')
hourly_shifts_by_user_df.tail()

user,AlanAzuelos,AlexGuttman,AlexanderHart,AlexandreDostaler,AlexisHaligua,AmelieBellemare,ArzuChaudhry,BernardUnger,DahliaGuttman,DanielMankarios,...,PaulBrisebois,PhilipStasiak,RafaelAroutiunian,SaraAhronheim,ShuoPeng,StephenRosenthal,TanLe,ThuHangTran,WayneChoi,WillGrad
ds,,,,,,,,,,,,,,,,,,,,,
2026-02-25 03:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-25 04:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-25 05:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-25 06:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-25 07:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking


In [8]:
df.columns.tolist()

['ds',
 'INFLOW_STRETCHER',
 'Infl_Stretcher_cum',
 'INFLOW_AMBULATORY',
 'Infl_Ambulatory_cum',
 'Inflow_Total',
 'Inflow_Cum_Total',
 'INFLOW_AMBULANCES',
 'Infl_Ambulances_cum',
 'FLS',
 'CUM_ADMREQ',
 'CUM_BA1',
 'WAITINGADM',
 'TTStr',
 'TRG_HALLWAY1',
 'TRG_HALLWAY_TBS',
 'reoriented_cum',
 'reoriented_cum_MD',
 'QTRACK1',
 'RESUS',
 'Pod_T',
 'POD_GREEN',
 'POD_GREEN_TBS',
 'POD_YELLOW',
 'POD_YELLOW_TBS',
 'POD_ORANGE',
 'POD_ORANGE_TBS',
 'POD_CONS_MORE2H',
 'POD_IMCONS_MORE4H',
 'POD_XRAY_MORE2H',
 'POD_CT_MORE2H',
 'POST_POD1',
 'VERTSTRET',
 'RAZ_TBS',
 'RAZ_LAZYBOY',
 'RAZ_WAITINGREZ',
 'AMBVERT1',
 'AMBVERTTBS',
 'QTrack_TBS',
 'Garage_TBS',
 'RAZ_CONS_MORE2H',
 'RAZ_IMCONS_MORE4H',
 'RAZ_XRAY_MORE2H',
 'RAZ_CT_MORE2H1',
 'PSYCH1',
 'PSYCH_WAITINGADM',
 'total_tbs',
 'vert_tbs',
 'pod_tbs',
 'overflow',
 'id']

In [9]:
ID_COL = "id"
TS_COL = "ds"
# TARGETS = ['total_tbs', 'Inflow_Total', 'overflow']
# Targets are all columns in df except ds (timestamp) and id
TARGETS = [col for col in df.columns.tolist() if col != TS_COL and col != ID_COL]

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates(
    [ID_COL, TS_COL], keep="last")





# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")

In [10]:
# Predict
print('Predicting basic forecast')
basic_forecast = pipeline.predict_df(
    df,
    prediction_length=24,
    # future_df = future_df.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

basic_forecast.head()

Predicting basic forecast


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,jgh,2026-02-16 22:00:00,INFLOW_STRETCHER,4.931792,2.550281,3.304924,3.873403,4.401141,4.931792,5.508318,6.109271,6.856080,7.908432
1,jgh,2026-02-16 23:00:00,INFLOW_STRETCHER,4.285006,1.989657,2.695039,3.243166,3.759193,4.285006,4.850069,5.446050,6.193225,7.259628
2,jgh,2026-02-17 00:00:00,INFLOW_STRETCHER,3.500295,1.505828,2.111206,2.589938,3.036917,3.500295,3.988601,4.536543,5.272153,6.337956
3,jgh,2026-02-17 01:00:00,INFLOW_STRETCHER,2.978490,1.128838,1.647893,2.089049,2.532100,2.978490,3.440379,3.938781,4.617775,5.592609
4,jgh,2026-02-17 02:00:00,INFLOW_STRETCHER,2.326559,0.848000,1.277569,1.639782,1.976918,2.326559,2.731115,3.173590,3.778267,4.704576


In [11]:
basic_forecast.target_name.value_counts()

target_name
INFLOW_STRETCHER       24
Infl_Stretcher_cum     24
INFLOW_AMBULATORY      24
Infl_Ambulatory_cum    24
Inflow_Total           24
Inflow_Cum_Total       24
INFLOW_AMBULANCES      24
Infl_Ambulances_cum    24
FLS                    24
CUM_ADMREQ             24
CUM_BA1                24
WAITINGADM             24
TTStr                  24
TRG_HALLWAY1           24
TRG_HALLWAY_TBS        24
reoriented_cum         24
reoriented_cum_MD      24
QTRACK1                24
RESUS                  24
Pod_T                  24
POD_GREEN              24
POD_GREEN_TBS          24
POD_YELLOW             24
POD_YELLOW_TBS         24
POD_ORANGE             24
POD_ORANGE_TBS         24
POD_CONS_MORE2H        24
POD_IMCONS_MORE4H      24
POD_XRAY_MORE2H        24
POD_CT_MORE2H          24
POST_POD1              24
VERTSTRET              24
RAZ_TBS                24
RAZ_LAZYBOY            24
RAZ_WAITINGREZ         24
AMBVERT1               24
AMBVERTTBS             24
QTrack_TBS             24


In [12]:
# basic_forecast


df_with_holidays = add_holiday_flags(df, ts_col='ds', include_names=True)

#create a dataframe with the next 24 hours timestamps hourly as column 'ds', with column 'id' jgh
future_df = hourly_shifts_by_user_df.reset_index()[hourly_shifts_by_user_df.reset_index()['ds'] > df['ds'].max()]
future_df['id'] = 'jgh'
future_df = add_holiday_flags(future_df, ts_col='ds', include_names=True)

# First, add holiday flags to future_df
future_df_with_added_holidays = add_holiday_flags(future_df, ts_col='ds', include_names=True)

# Then, select only the columns from future_df_with_added_holidays that are also in df_with_holidays
common_columns = [col for col in future_df_with_added_holidays.columns if col in df_with_holidays.columns]
future_df_with_holidays = future_df_with_added_holidays[common_columns]

# Predict
print('Predicting forecast with holidays')  
forecast_with_holidays = pipeline.predict_df(
    df_with_holidays,
    prediction_length=24,
    future_df = future_df_with_holidays.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
forecast_with_holidays.tail()

Predicting forecast with holidays


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1171,jgh,2026-02-17 17:00:00,overflow,10.468918,3.125813,5.385239,7.303908,8.952517,10.468918,12.091251,13.795901,15.866001,18.665737
1172,jgh,2026-02-17 18:00:00,overflow,9.176695,2.527975,4.511410,6.230162,7.754889,9.176695,10.637086,12.143537,14.021288,16.714180
1173,jgh,2026-02-17 19:00:00,overflow,8.213021,2.150874,4.021453,5.601889,6.961201,8.213021,9.487517,10.823153,12.512251,14.992601
1174,jgh,2026-02-17 20:00:00,overflow,7.280141,1.756843,3.481067,4.903116,6.120342,7.280141,8.469582,9.705862,11.243631,13.479563
1175,jgh,2026-02-17 21:00:00,overflow,6.434717,1.455413,3.012400,4.293056,5.391843,6.434717,7.547089,8.714396,10.190226,12.351694


In [13]:
df_with_staffing = df.merge(hourly_shifts_by_user_df, on='ds')
future_df_with_staffing = hourly_shifts_by_user_df.reset_index()[hourly_shifts_by_user_df.reset_index()['ds'] > df['ds'].max()]
future_df_with_staffing['id'] = 'jgh'

print('Predicting forecast with staffing')
forecast_with_staffing = pipeline.predict_df(
    df_with_staffing,
    prediction_length=24,
    future_df = future_df_with_staffing.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

forecast_with_staffing.tail()

Predicting forecast with staffing


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1171,jgh,2026-02-17 17:00:00,overflow,10.829508,2.758563,5.166518,7.278488,9.154808,10.829508,12.710493,14.754307,17.255810,20.454113
1172,jgh,2026-02-17 18:00:00,overflow,9.157551,2.239035,4.271985,6.061616,7.683002,9.157551,10.771079,12.659819,15.097538,18.293949
1173,jgh,2026-02-17 19:00:00,overflow,8.360128,2.241612,4.132365,5.675675,7.045011,8.360128,9.783594,11.307749,13.256687,16.127224
1174,jgh,2026-02-17 20:00:00,overflow,7.635161,2.057368,3.821157,5.187780,6.402361,7.635161,8.937738,10.297767,12.003635,14.543182
1175,jgh,2026-02-17 21:00:00,overflow,6.533813,1.500557,3.113539,4.333365,5.428456,6.533813,7.737551,9.019010,10.671433,13.194708


In [14]:
weather_df = pd.read_csv('https://www.dropbox.com/scl/fi/gmhwwld9z9yychg4r0yuk/weather.csv?rlkey=66c78m90aviamr0x0uu72pfr8&raw=1')
weather_df.ds = pd.to_datetime(weather_df.ds, errors="coerce")


future_weather_df = weather_df[weather_df.ds > df.ds.max()].head(24)
future_weather_df['id']='jgh'

print('Predicting forecast with weather')
# Predict
forecast_with_weather = pipeline.predict_df(
    #join df with weather_df on ds
    df.merge(weather_df, on='ds'),
    prediction_length=24,
    #weather_df where ds is greater than the max of df.ds.max()
    future_df = future_weather_df,
    # future_df = future_df.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

Predicting forecast with weather


In [15]:
weather_df.tail()

,ds,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,wind_speed_10m,wind_direction_10m,wind_gusts_10m,precipitation_probability
45096,2026-02-23 00:00:00,-6.611,91.0,-7.835336,-11.807478,0.0,0.0,0.0,0.27,3.0,1016.099976,987.167664,100.0,50.0,100.0,100.0,14.578890,32.905247,39.239998,16.0
45097,2026-02-23 01:00:00,-7.011,91.0,-8.231208,-12.287523,0.0,0.0,0.0,0.27,3.0,1015.799988,986.833679,100.0,53.0,100.0,100.0,14.882420,32.152271,39.599998,18.0
45098,2026-02-23 02:00:00,-7.461,91.0,-8.676571,-12.794329,0.0,0.0,0.0,0.27,3.0,1015.500000,986.493713,100.0,63.0,100.0,100.0,15.003839,30.256361,39.239998,18.0
45099,2026-02-23 03:00:00,-7.811,91.0,-9.022971,-13.219419,0.0,0.0,0.0,0.27,3.0,1015.200012,986.164673,100.0,71.0,100.0,100.0,15.315873,29.577742,38.519997,18.0
45100,2026-02-23 04:00:00,-7.911,91.0,-9.121943,-13.309154,0.0,0.0,0.0,0.27,3.0,1014.900024,985.862793,100.0,72.0,100.0,100.0,15.188416,31.429514,38.160000,18.0


In [16]:
# All variables forecast
print('Predicting all variables forecast')
all_variable_df = add_holiday_flags(df_with_staffing, ts_col='ds', include_names=True).merge(weather_df, on='ds')

forecast_all_vars_with_future = pipeline.predict_df(
    all_variable_df,
    prediction_length=24,
    #future_df should be future_df_with_staffing merged with future_weather_df on 'ds' and 'id'
    future_df = future_df_with_staffing.merge(future_weather_df, on=['ds', 'id']),
    # quantile_levels=[0.1, 0.5, 0.9],
    quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)   
forecast_all_vars_with_future.tail()

Predicting all variables forecast


,id,ds,target_name,predictions,0.5
1171,jgh,2026-02-17 17:00:00,overflow,10.365742,10.365742
1172,jgh,2026-02-17 18:00:00,overflow,8.921942,8.921942
1173,jgh,2026-02-17 19:00:00,overflow,7.992727,7.992727
1174,jgh,2026-02-17 20:00:00,overflow,7.276100,7.276100
1175,jgh,2026-02-17 21:00:00,overflow,6.142009,6.142009


In [17]:
#join the predictions columns of basic_forecast, forecast_with_holidays, forecast_with_staffing, forecast_with_weather, forecast_all_vars_without_future, forecast_all_vars_with_future on the 'ds' column
basic_forecast = basic_forecast[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'basic_forecast'})
forecast_with_holidays = forecast_with_holidays[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_holidays'})
forecast_with_staffing = forecast_with_staffing[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_staffing'})
forecast_with_weather = forecast_with_weather[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_weather'})
# forecast_all_vars_without_future = forecast_all_vars_without_future[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_all_vars_without_future'})
forecast_all_vars_with_future = forecast_all_vars_with_future[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_all_vars_with_future'})

pred_df = basic_forecast.merge(forecast_with_holidays, on=['ds', 'target_name']).merge(forecast_with_staffing, on=['ds', 'target_name']).merge(forecast_with_weather, on=['ds', 'target_name']).merge(forecast_all_vars_with_future, on=['ds', 'target_name'])
pred_df.head()

,ds,target_name,basic_forecast,forecast_with_holidays,forecast_with_staffing,forecast_with_weather,forecast_all_vars_with_future
0,2026-02-16 22:00:00,INFLOW_STRETCHER,4.931792,4.896488,5.050235,5.003682,4.979941
1,2026-02-16 23:00:00,INFLOW_STRETCHER,4.285006,4.270098,4.691194,4.185825,4.491570
2,2026-02-17 00:00:00,INFLOW_STRETCHER,3.500295,3.515881,4.051642,3.383240,3.745417
3,2026-02-17 01:00:00,INFLOW_STRETCHER,2.978490,3.005519,3.530185,2.913721,3.200379
4,2026-02-17 02:00:00,INFLOW_STRETCHER,2.326559,2.355932,2.593929,2.311968,2.432643


In [18]:
# Create a new dataframe with the average % difference of each forecast compared to the basic forecast, for each target_name averaged over all ds
# The output dataframe should only have 1 row per target_name, and columns for the average % difference of each forecast compared to the basic forecast
comparison_df = pred_df.copy()
comparison_df['%diff_holidays'] = (comparison_df['forecast_with_holidays'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_staffing'] = (comparison_df['forecast_with_staffing'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_weather'] = (comparison_df['forecast_with_weather'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_all_vars_with_future'] = (comparison_df['forecast_all_vars_with_future'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df = comparison_df.groupby('target_name')[['%diff_holidays', '%diff_staffing', '%diff_weather', '%diff_all_vars_with_future']].mean().reset_index()
comparison_df.head()
# comparison_df.to_csv('forecast_variable_effects.csv', index=False)

,target_name,%diff_holidays,%diff_staffing,%diff_weather,%diff_all_vars_with_future
0,AMBVERT1,0.297092,-0.906799,3.819360,4.268310
1,AMBVERTTBS,0.677798,1.770922,6.475342,8.760206
2,CUM_ADMREQ,-0.025748,12.883042,-0.969341,8.798456
3,CUM_BA1,-0.052889,20.572300,-6.320881,12.150628
4,FLS,81.263954,289.654846,35.505547,316.729279


In [19]:
anomaly_detection_ranges_df = pd.read_csv('https://www.dropbox.com/scl/fi/fjz0am427gw35sz7l994m/anomaly_detection_ranges.csv?rlkey=lib9w0jz2zei5n566jv76o7ol&raw=1')
anomaly_detection_ranges_df.ds = pd.to_datetime(anomaly_detection_ranges_df.ds, errors="coerce")
anomaly_detection_ranges_df.tail()

,ds,INFLOW_STRETCHER_yhat,INFLOW_STRETCHER_yhat_lower,INFLOW_STRETCHER_yhat_upper,Infl_Stretcher_cum_yhat,Infl_Stretcher_cum_yhat_lower,Infl_Stretcher_cum_yhat_upper,INFLOW_AMBULATORY_yhat,INFLOW_AMBULATORY_yhat_lower,INFLOW_AMBULATORY_yhat_upper,...,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper,overflow_yhat,overflow_yhat_lower,overflow_yhat_upper
331,2026-02-17 10:00:00,8.595712,3.554119,13.517478,47.723544,13.078714,82.473518,8.852669,4.014644,13.442297,...,37.956685,16.995280,2.559104,32.859980,3.707365,-1.113128,8.724998,12.282530,4.211947,20.290898
332,2026-02-17 11:00:00,9.687093,4.539101,14.101765,60.644536,24.332223,96.551032,9.295951,4.693106,14.056750,...,39.226278,18.479611,3.080171,32.434850,4.451797,-0.519434,9.610966,12.775318,4.478152,20.883682
333,2026-02-17 12:00:00,10.284545,5.329003,15.114360,68.020177,30.113809,106.320080,8.912751,4.274332,13.676663,...,42.830903,20.955425,5.945452,35.394854,5.336680,0.503265,10.277198,13.486158,5.763390,21.540793
334,2026-02-17 13:00:00,10.425223,5.926774,15.152485,72.147128,32.129208,106.426742,8.283940,3.335691,12.904704,...,47.741766,24.373396,9.681965,38.839372,6.233579,1.234512,11.036601,14.339756,6.060332,22.118518
335,2026-02-17 14:00:00,10.167426,4.909505,15.298731,79.373912,41.276287,118.178607,7.814351,3.365244,12.515837,...,51.090844,27.700490,12.294372,42.764633,6.954979,1.849109,12.033232,15.129058,6.429588,22.820212


In [24]:
recent_df = df.tail(24)

In [30]:
targets = pred_df['target_name'].unique().tolist()
output_df = pd.DataFrame()
for target in targets:
    target_df = pred_df[pred_df['target_name'] == target][['ds', 'forecast_all_vars_with_future']].rename(columns={'forecast_all_vars_with_future': target+'_forecast'})
    target_df = target_df.merge(anomaly_detection_ranges_df[['ds', target+'_yhat', target+'_yhat_lower', target+'_yhat_upper']], on=['ds'], how='left')
    target_df[target+'_anomaly'] = ((target_df[target+'_forecast'] < target_df[target+'_yhat_lower']) | (target_df[target+'_forecast'] > target_df[target+'_yhat_upper'])).map({True: 'yes', False: 'no'})
    #assign a colour based on how the value compares to the yhat and the yhat_lower and yhat_upper. If it's an anomaly, colour is #D13438. If it's between yhat and yhat_upper, colour is #FFB900. If it's between yhat_lower and yhat, colour is #107C10. 
    target_df[target+'_colour'] = target_df.apply(lambda row: '#D13438' if row[target+'_anomaly'] == 'yes' else ('#FFB900' if row[target+'_forecast'] > row[target+'_yhat'] else '#107C10'), axis=1)
    if output_df.empty:
        output_df = target_df
    else:
        output_df = output_df.merge(target_df, on='ds', how='outer')

output_df = output_df.merge(recent_df.tail(24), on='ds', how='outer')
output_df.head()

output_df.to_csv('ED_Hourly_Forecasts_Anomalies_v1.0.csv', index=False)
    

In [28]:
output_df.merge(recent_df.tail(24), on='ds', how='outer')[['ds', 'total_tbs', 'total_tbs_forecast', 'total_tbs_yhat', 'total_tbs_yhat_lower', 'total_tbs_yhat_upper', 'total_tbs_anomaly', 'total_tbs_colour']]

,ds,total_tbs,total_tbs_forecast,total_tbs_yhat,total_tbs_yhat_lower,total_tbs_yhat_upper,total_tbs_anomaly,total_tbs_colour
0,2026-02-15 22:00:00,17.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2026-02-15 23:00:00,15.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-02-16 00:00:00,16.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-02-16 01:00:00,18.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2026-02-16 02:00:00,16.0,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-02-16 03:00:00,14.0,NaN,NaN,NaN,NaN,NaN,NaN
6,2026-02-16 04:00:00,12.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2026-02-16 05:00:00,9.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-02-16 06:00:00,5.0,NaN,NaN,NaN,NaN,NaN,NaN
9,2026-02-16 07:00:00,5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
dropbox_app_key = os.environ.get("DROPBOX_APP_KEY")
dropbox_app_secret = os.environ.get("DROPBOX_APP_SECRET")
dropbox_refresh_token = os.environ.get("DROPBOX_REFRESH_TOKEN")

# exchange the authorization code for an access token:
token_url = "https://api.dropboxapi.com/oauth2/token"
params = {
    "grant_type": "refresh_token",
    "refresh_token": dropbox_refresh_token,
    "client_id": dropbox_app_key,
    "client_secret": dropbox_app_secret
}
r = requests.post(token_url, data=params)

dropbox_access_token = r.json()['access_token']

dbx = dropbox.Dropbox(dropbox_access_token)

upload(dbx, 'chronos_forecast.csv', '', '',
            'chronos_forecast.csv', overwrite=True)
upload(dbx, 'ED_Hourly_Forecasts_Anomalies_v1.0.csv', '', '',
            'ED_Hourly_Forecasts_Anomalies_v1.0.csv', overwrite=True)
upload(dbx, 'forecast_variable_effects.csv', '', '',
            'forecast_variable_effects.csv', overwrite=True)

uploaded as b'chronos_forecast.csv'
uploaded as b'ED_Hourly_Forecasts_Anomalies_v1.0.csv'
uploaded as b'forecast_variable_effects.csv'


FileMetadata(client_modified=datetime.datetime(2026, 2, 16, 20, 46, 58), content_hash='e67091084ccd8e1bdd5edfb96548e0e89d3c74f2d3983d49269226560633e641', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:oNSmVCFixyQAAAAAAABxQQ', is_downloadable=True, media_info=NOT_SET, name='forecast_variable_effects.csv', parent_shared_folder_id=NOT_SET, path_display='/forecast_variable_effects.csv', path_lower='/forecast_variable_effects.csv', preview_url=NOT_SET, property_groups=NOT_SET, rev='64afc60eaac237a19c0a3', server_modified=datetime.datetime(2026, 2, 17, 3, 7, 58), sharing_info=NOT_SET, size=2710, symlink_info=NOT_SET)

In [111]:
print(pred_df[pred_df['target_name'] == 'total_tbs'][['ds', 'basic_forecast', 'forecast_with_holidays', 'forecast_with_staffing', 'forecast_with_weather', 'forecast_all_vars_with_future']].head(30))

                      ds  basic_forecast  forecast_with_holidays  \
1080 2026-02-16 16:00:00       43.248169               43.369469   
1081 2026-02-16 17:00:00       36.762802               36.773109   
1082 2026-02-16 18:00:00       32.573738               32.730572   
1083 2026-02-16 19:00:00       27.893730               28.034868   
1084 2026-02-16 20:00:00       26.208014               26.412586   
1085 2026-02-16 21:00:00       23.475424               23.650637   
1086 2026-02-16 22:00:00       19.563084               19.743649   
1087 2026-02-16 23:00:00       18.025482               18.188704   
1088 2026-02-17 00:00:00       18.764906               18.898991   
1089 2026-02-17 01:00:00       18.195299               18.347420   
1090 2026-02-17 02:00:00       16.028679               16.149977   
1091 2026-02-17 03:00:00       14.295581               14.424920   
1092 2026-02-17 04:00:00       12.925599               13.071001   
1093 2026-02-17 05:00:00       10.454781        

In [23]:
df.tail(24)

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs,overflow,id
44807,2026-02-15 22:00:00,4,110,1,77,5,187,0,30,0,...,0,1,0,7,6,17,14,3,3,jgh
44808,2026-02-15 23:00:00,9,119,1,78,10,197,1,31,1,...,0,0,0,7,6,15,12,3,4,jgh
44809,2026-02-16 00:00:00,4,123,1,79,5,202,2,33,1,...,0,0,0,9,6,16,11,5,4,jgh
44810,2026-02-16 01:00:00,3,3,3,3,6,6,1,1,1,...,0,0,0,10,6,18,15,3,4,jgh
44811,2026-02-16 02:00:00,1,4,0,3,1,7,0,1,0,...,0,0,0,10,6,16,14,2,4,jgh
44812,2026-02-16 03:00:00,2,6,0,3,2,9,2,3,0,...,0,0,0,10,6,14,11,3,4,jgh
44813,2026-02-16 04:00:00,5,11,2,5,7,16,2,5,0,...,0,0,0,11,6,12,11,1,4,jgh
44814,2026-02-16 05:00:00,0,11,1,6,1,17,0,5,0,...,0,0,0,11,6,9,6,3,4,jgh
44815,2026-02-16 06:00:00,1,12,0,6,1,18,0,5,0,...,1,0,0,11,6,5,3,2,2,jgh
44816,2026-02-16 07:00:00,3,15,2,8,5,23,1,6,0,...,0,0,0,11,6,5,4,1,0,jgh
